In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/PyAI'

/content/drive/MyDrive/PyAI


In [3]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [4]:
train = datasets.MNIST(root = "/data", train = True, transform=transforms.ToTensor(), download = True)
test = datasets.MNIST(root = "/data", train = False, transform=transforms.ToTensor(), download = True)

train_data = DataLoader(train, batch_size = 100, shuffle = True)
test_data = DataLoader(test, batch_size = 1000, shuffle = False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 17.2MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 477kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.40MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 10.4MB/s]


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

#x, t 입력
#dataloader에서 처리

#함수 정의
class NN(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.f = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5), #(1*28*28) -> (16*24*24)
            nn.ReLU(),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5), #(16*24*24) -> (32*20*20)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2), #(32*20*20) -> (32*10*10)
            nn.Conv2d(in_channels=32,out_channels=64,kernel_size=5), #(32*10*10) -> (64*6*6)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2), #(64*6*6) -> (64*3*3)
        )
        self.g = nn.Sequential(
            nn.Linear(64*3*3,100),
            nn.ReLU(),
            nn.Linear(100,10),
        )
    def forward(self,x) :
        x = self.f(x)
        x = x.reshape(x.shape[0], -1)
        x = self.g(x)
        return x
F = NN()
F = F.to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(F.parameters(), lr = 0.002)
epoch = 10
prev_acc = 0
cnt = 0

for e in range(epoch) :
    loss_sum = 0
    for x, t in train_data :
#순전파
        y = F(x.to(device))
#손실함수
        loss = loss_func(y, t.to(device))
        loss_sum += loss
#역전파
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    loss_sum /= len(train_data)

    correct = 0
    total = 0
    for x, t in test_data :
        with torch.no_grad() :
            y = F(x.to(device))
        correct += (y.argmax(dim = 1) == t.to(device)).sum().item()
        total += len(x)
    acc = correct / total

    if acc <= prev_acc + 0.001 :
        cnt += 1
    else :
        cnt = 0
        prev_acc = acc
    print(f"epoch {e+1} | loss {loss_sum} | acc {acc}")
    if cnt >= 3 :
        print("train halted")
        break

epoch 1 | loss 0.18391992151737213 | acc 0.9809
epoch 2 | loss 0.049904897809028625 | acc 0.9893
epoch 3 | loss 0.0363864004611969 | acc 0.9908
epoch 4 | loss 0.028935004025697708 | acc 0.9922
epoch 5 | loss 0.023714791983366013 | acc 0.9899
epoch 6 | loss 0.020570900291204453 | acc 0.9904
epoch 7 | loss 0.018853971734642982 | acc 0.99
train halted


In [7]:
cnt = 0;
total = 0
err = []

for x, t in test_data :
    y = F(x.to(device))
    for i in range(x.shape[0]) :
        if torch.argmax(y[i]) == t.to(device)[i] :
            cnt += 1
        else :
            err.append(total)
        total += 1

print("correct_posibility : %f" %(cnt / total))
print(err)
torch.save(F.to("cpu"),"MNIST.pt")

correct_posibility : 0.990000
[115, 158, 320, 321, 445, 449, 543, 582, 625, 646, 659, 882, 883, 947, 1014, 1033, 1039, 1112, 1114, 1178, 1232, 1242, 1247, 1260, 1299, 1394, 1500, 1530, 1681, 1737, 1790, 1828, 1901, 2035, 2053, 2070, 2130, 2135, 2189, 2293, 2329, 2447, 2597, 2607, 2654, 2896, 2921, 2927, 2939, 2953, 3023, 3108, 3225, 3288, 3422, 3448, 3474, 3475, 3574, 3762, 3767, 3796, 3976, 4018, 4027, 4078, 4224, 4437, 4443, 4504, 4571, 4639, 4740, 4761, 4783, 4860, 5450, 5734, 5937, 5997, 6571, 6572, 6576, 6597, 6625, 8059, 8061, 8408, 8520, 8527, 9009, 9015, 9638, 9642, 9664, 9692, 9729, 9792, 9839, 9874]
